In [1]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt

In [2]:
import tensorflow

In [3]:
import sys
print(sys.executable)

C:\Program Files\Python38\python.exe


In [4]:
import pandas as pd

In [5]:
# from google.colab import drive
# drive.mount('/content/drive')

In [6]:
# import os
# os.chdir("/content/drive/")

In [7]:
# %cd MyDrive/Colab Notebooks/Room Classification

In [8]:
TRAINING_PATH = 'rest_tagging-master/Dataset/'

In [9]:
subfolders = os.listdir(TRAINING_PATH)
# subfolders.remove('Interior')

In [10]:
subfolders

['bathroom',
 'bedroom',
 'dining_room',
 'Exterior',
 'Interior',
 'kitchen',
 'living_room']

In [11]:
def preprocessing(batch_folder, batch_id):
    X = []
    y = []
    for j in range(len(subfolders)):
        count = 0;
        files = os.listdir(TRAINING_PATH + subfolders[j])
        index_lst = np.random.randint(len(files), size=1000)
        for idx in index_lst:
            img = cv2.imread(TRAINING_PATH + subfolders[j] + '/' + files[idx]).astype(np.float32)
            img_resized = cv2.resize(img, (240,240,))/255
            X.append(img_resized)
            y.append(j)
            count += 1
            if (count == 1000):
                print('     Processing ' + subfolders[j] +' samples: Done' )
                break
    X = np.array(X, copy=False)
    y = np.array(y, copy=False)
    batch_path = batch_folder + '/batch_' + str(batch_id) + '.npz' 
    np.savez(batch_path, X=X, y=y)
    print('   >> Batch ' + str(batch_id) + ' saved.')

In [12]:
def preprocess_and_save_bootstrap_batches(batch_folder, num_batch):
    os.mkdir(batch_folder)
    for i in range(num_batch):
        print('------------------------------------------')
        print('Preprocessing batch ' + str(i+1) + 'th:')
        preprocessing(batch_folder, i+1)

In [13]:
preprocess_and_save_bootstrap_batches('batches', 10)

------------------------------------------
Preprocessing batch 1th:
     Processing bathroom samples: Done
     Processing bedroom samples: Done
     Processing dining_room samples: Done
     Processing Exterior samples: Done
     Processing Interior samples: Done
     Processing kitchen samples: Done
     Processing living_room samples: Done
   >> Batch 1 saved.
------------------------------------------
Preprocessing batch 2th:
     Processing bathroom samples: Done
     Processing bedroom samples: Done
     Processing dining_room samples: Done
     Processing Exterior samples: Done
     Processing Interior samples: Done
     Processing kitchen samples: Done
     Processing living_room samples: Done
   >> Batch 2 saved.
------------------------------------------
Preprocessing batch 3th:
     Processing bathroom samples: Done
     Processing bedroom samples: Done
     Processing dining_room samples: Done
     Processing Exterior samples: Done
     Processing Interior samples: Done
   

In [ ]:
def get_Xy_test(model_folder, batch_folder, batch_id, X_test_lst, y_test_lst):
    batch_path = batch_folder + '/batch_' + str(batch_id) + '.npz' 
    with np.load(batch_path) as batch:
        X = batch['X']
        y = batch['y']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_test_lst.append(X_test.tolist())
    y_test_lst.append(y_test.tolist())

In [ ]:
X_test = []
y_test = []
for i in range(1, 6):
    get_Xy_test('bagging_models', 'batches', i, X_test, y_test)

In [ ]:
X_test = np.array(X_test, copy=False)
y_test = np.array(y_test, copy=False)

In [ ]:
X_test = np.reshape(X_test, (1400*5, 240, 240, 3))
y_test = np.reshape(y_test, (1400*5))

In [ ]:
np.savez('batches/test_set.npz', X_test=X_test, y_test=y_test)